In [7]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
torch.set_printoptions(linewidth=120)

In [8]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
    
    def forward(self, t):
        # hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # output layer
        t = self.out(t)
        #t = F.softmax(t, dim=1)

        return t

Wir erstellen das train_set.

In [9]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

#### Der Forward-Pass für ein Bild

In [17]:
image, label  = next(iter(train_set)) 
image.shape, label

(torch.Size([1, 28, 28]), 9)

Ein einzelnes Bild können wir als Batch der Länge 1 auffassen. Wir bringen den Eingabe-Tensor in das gewünschte Format

In [18]:
image.unsqueeze(0).shape

torch.Size([1, 1, 28, 28])

In [19]:
net = Network()
pred = net(image.unsqueeze(0))
pred

tensor([[ 0.0718,  0.1010, -0.0319, -0.0910, -0.0869,  0.0176, -0.0334, -0.1134, -0.1003,  0.0897]],
       grad_fn=<AddmmBackward>)

Die Softmax-Funktion normiert die letzte Ausgabe so, dass sie als Wahrscheinlichkeiten gelesen werden können.

In [20]:
pred.sum()

tensor(-0.1768, grad_fn=<SumBackward0>)

Wir suchen den Index mit der größten Wahrscheinlichkeit und vergleichen ihn mit dem Label.

In [21]:
pred.argmax().item(), label

(1, 9)

#### Der Forward-Pass für einen Batch

In [22]:
train_loader = torch.utils.data.DataLoader(train_set
    ,batch_size=10
)
batch = iter(train_loader)
images, labels = next(batch)
images.shape, labels

(torch.Size([10, 1, 28, 28]), tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5]))

In [23]:
preds = net(images)
preds.shape, preds

(torch.Size([10, 10]),
 tensor([[ 0.0718,  0.1010, -0.0319, -0.0910, -0.0869,  0.0176, -0.0334, -0.1134, -0.1003,  0.0897],
         [ 0.0567,  0.0951, -0.0375, -0.0988, -0.0845,  0.0174, -0.0302, -0.1035, -0.1008,  0.0916],
         [ 0.0576,  0.0953, -0.0428, -0.0939, -0.0824, -0.0052, -0.0239, -0.1076, -0.1023,  0.0861],
         [ 0.0597,  0.0954, -0.0412, -0.0922, -0.0847, -0.0033, -0.0301, -0.1139, -0.1034,  0.0877],
         [ 0.0642,  0.0942, -0.0353, -0.0911, -0.0879,  0.0011, -0.0394, -0.1177, -0.1016,  0.0948],
         [ 0.0608,  0.0975, -0.0333, -0.0949, -0.0858,  0.0178, -0.0335, -0.1094, -0.1031,  0.0868],
         [ 0.0526,  0.0952, -0.0358, -0.0988, -0.0828,  0.0052, -0.0289, -0.1116, -0.1027,  0.0829],
         [ 0.0640,  0.0950, -0.0340, -0.0949, -0.0791,  0.0186, -0.0278, -0.1034, -0.1104,  0.0817],
         [ 0.0650,  0.0970, -0.0329, -0.0897, -0.0794,  0.0019, -0.0256, -0.1048, -0.0971,  0.0857],
         [ 0.0665,  0.0943, -0.0272, -0.1087, -0.0864,  0.0335, -0.0

Wir vergleichen die Vorhersage mit den Labels

In [24]:
preds.argmax(dim=1), labels

(tensor([1, 1, 1, 1, 9, 1, 1, 1, 1, 1]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5]))

Wo stimmen die Werte überein?

In [25]:
preds.argmax(dim=1).eq(labels) 

tensor([False, False, False, False, False, False, False, False, False, False])

Anzahl der Übereinstimmungen

In [26]:
preds.argmax(dim=1).eq(labels).sum()

tensor(0)

In [27]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

get_num_correct(preds, labels)

0

#### Loss, Gradient, Optimizer

In [36]:
loss = F.cross_entropy(preds, labels) 
loss.item()

tensor(2.3034, grad_fn=<NllLossBackward>)

In [37]:
print(net.conv1.weight.grad)

AttributeError: 'NoneType' object has no attribute 'shape'

In [39]:
loss.backward()

In [40]:
print(net.conv1.weight.grad.shape)
print(net.conv1.weight.grad)

torch.Size([6, 1, 5, 5])
tensor([[[[ 1.5791e-06,  3.0987e-06, -2.3689e-06, -2.9477e-06, -1.2152e-05],
          [ 1.2617e-08, -1.6410e-07, -4.5328e-06, -1.6404e-05, -9.0226e-06],
          [-6.5153e-07, -8.2259e-07, -1.5639e-05, -1.5979e-05,  3.9226e-06],
          [-5.3651e-07, -5.4110e-06, -2.0092e-05, -1.6125e-06,  2.3402e-06],
          [-1.1005e-06, -1.2314e-05, -1.6940e-05,  3.5824e-07, -4.3728e-06]]],


        [[[-5.9894e-05, -1.2090e-04, -5.0924e-05,  9.6953e-06, -1.4242e-04],
          [-3.0523e-05, -2.1103e-05, -4.2214e-05,  2.8891e-05, -1.1105e-04],
          [ 6.7964e-05,  7.7582e-05,  5.8374e-05,  2.7225e-06, -2.1703e-04],
          [ 1.4244e-04,  6.1349e-05,  1.6056e-05, -2.6335e-05, -2.6858e-04],
          [ 1.6518e-04,  1.5072e-04, -1.6581e-05,  3.6636e-06, -2.2057e-04]]],


        [[[ 2.0203e-07,  2.9694e-07,  8.1515e-07, -1.7188e-08,  2.3142e-07],
          [ 3.0464e-08,  3.0464e-08,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 1.0385e-06,  2.5008e-06,  3.892

Wir wählen einen Optimizer und ihn die Gewichte updaten. Danach machen wir wieder eine prediction.

In [46]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
optimizer.step()  
preds = net(images)

loss = F.cross_entropy(preds, labels)
print('loss:', loss.item())
get_num_correct(preds, labels)

loss: 2.1882872581481934


3

Dreimal der gesamte Ablauf mit einer Batch-Size von 100:

In [55]:
net = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(net.parameters(), lr=0.01)

batch = next(iter(train_loader)) # Get Batch
images, labels = batch

preds = net(images)  
loss = F.cross_entropy(preds, labels)  
print('loss1:', loss.item(), 'num correct:', get_num_correct(preds,labels))

loss.backward()  
optimizer.step()  
preds = net(images)
loss = F.cross_entropy(preds, labels)
print('loss2:', loss.item(), 'num correct:', get_num_correct(preds,labels))

loss.backward()  
optimizer.step()  
preds = net(images)
loss = F.cross_entropy(preds, labels)
print('loss3:', loss.item(), 'num correct:', get_num_correct(preds,labels))

loss1: 2.302917003631592 num correct: 8
loss2: 2.299283027648926 num correct: 15
loss3: 2.28469181060791 num correct: 15


#### Eine Epoche

Wenn die Batche alle Daten des Trainingssets durchlaufen haben, ist eine *Epoche* zu Ende.

In [58]:
net = Network()

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)
optimizer = optim.Adam(net.parameters(), lr=0.01)

total_loss = 0
total_correct = 0

for batch in train_loader: # Get Batch
    images, labels = batch 

    preds = net(images) # Pass Batch
    loss = F.cross_entropy(preds, labels) # Calculate Loss

    optimizer.zero_grad()
    loss.backward() # Calculate Gradients
    optimizer.step() # Update Weights

    total_loss += loss.item()
    total_correct += get_num_correct(preds, labels)
    
print(
    "epoch:", 0, 
    "total_correct:", total_correct, 
    "loss:", total_loss
)

epoch: 0 total_correct: 47014 loss: 342.30467838048935


Prozentsatz der richtigen Zuordnungen nach einer Epoche

In [59]:
total_correct / len(train_set)

0.7835666666666666

#### Training mit mehreren Epochen

In [36]:
net = Network()
optimizer = optim.Adam(net.parameters(), lr=0.01)
train_loader = torch.utils.data.DataLoader(
     train_set
    ,batch_size=100
    ,shuffle=True
)

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, labels = batch 

        preds = net(images) # Pass Batch
        loss = F.cross_entropy(preds, labels) # Calculate Loss

        optimizer.zero_grad()
        loss.backward() # Calculate Gradients
        optimizer.step() # Update Weights

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print(
        "epoch", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )

epoch 0 total_correct: 46886 loss: 348.9371727705002
epoch 1 total_correct: 51228 loss: 240.15532341599464
epoch 2 total_correct: 52097 loss: 215.706786647439
epoch 3 total_correct: 52549 loss: 204.76445497572422
epoch 4 total_correct: 52653 loss: 198.21014676988125
epoch 5 total_correct: 52711 loss: 196.00613040477037
epoch 6 total_correct: 52696 loss: 198.06398537755013
epoch 7 total_correct: 53015 loss: 190.6323289424181
epoch 8 total_correct: 53302 loss: 181.53220679610968
epoch 9 total_correct: 53201 loss: 184.2184516787529


```
epoch 0 total_correct: 46886 loss: 348.9371727705002
epoch 1 total_correct: 51228 loss: 240.15532341599464
epoch 2 total_correct: 52097 loss: 215.706786647439
epoch 3 total_correct: 52549 loss: 204.76445497572422
epoch 4 total_correct: 52653 loss: 198.21014676988125
epoch 5 total_correct: 52711 loss: 196.00613040477037
epoch 6 total_correct: 52696 loss: 198.06398537755013
epoch 7 total_correct: 53015 loss: 190.6323289424181
epoch 8 total_correct: 53302 loss: 181.53220679610968
epoch 9 total_correct: 53201 loss: 184.2184516787529
correct- test: correct: 0.8747
```

#### Anwenden auf die Testdaten

In [37]:
test_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=False
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

test_loader = torch.utils.data.DataLoader(test_set
    ,batch_size=100
)

correct = 0
for batch in test_loader:
    images, labels = batch 
    preds = net(images)
    
    images.shape, labels
    correct += get_num_correct(preds,labels)
    
print("correct:",correct/len(test_set))

correct: 0.8747


#### Model und Parameter speichern und laden

In [63]:
checkpoint = {'model': Network(),
          'state_dict': net.state_dict(),
          'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

C:\Users\khthe\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\khthe\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\khthe\.conda\envs\pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [30]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False

    model.eval()
    return model

model = load_checkpoint('checkpoint.pth')

In [31]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

batch = next(iter(train_loader)) # Get Batch
images, labels = batch
preds = model(images) # Pass Batch
get_num_correct(preds, labels)
    

86